In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import scipy.misc
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
#import essentia, essentia.standard as ess

In [2]:
def chargen(char=20):
    print char
    char_gen = []
    i=0
    genre={}
    cont=0
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                #mfcc = np.ndarray.tolist(mfcc)

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
        print 
                    
    return np.array(char_gen)

In [3]:
def char1(char=5):
    print char
    char_gen=[]
    i=0
    genre={}
    cont=0
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                                
                #librosa.display.specshow(mfcc, x_axis='time')
                #plt.show()

                mfcc=np.transpose(mfcc)
                mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
                '''if cont>=10:
                    break'''
        print 
                    
    return np.array(char_gen)

In [4]:
def char2(char=5):
    char_gen=[]
    i=0
    genre={}
    for folder in os.listdir("./genres"): #iterates all folders    
        cont=0

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=30.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_fft=1024, hop_length=30*22050) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1922

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                
                #librosa.display.specshow(mfcc, x_axis='time')
                #plt.show()
                
                mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list

                cont+=1
                if cont>=1:
                    break
                
    return char_gen

In [5]:
def char3(char=5):
    print char
    char_gen=[]
    maxs=[]
    mins=[]
    i=0
    genre={}
    cont=0
    scaler = StandardScaler()
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        print str(cont/100)+":",    
            
        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                
                #mfcc = scaler.fit_transform(mfcc)
                                
                maxs.append(np.amax(mfcc))
                mins.append(np.amin(mfcc))
                
                #mfcc=np.transpose(mfcc)
                #mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
        print 
    step = (max(maxs) - min(mins))/100
    return np.array(char_gen), step

In [6]:
def histogram(step, char, amount):
    histogram = np.zeros(amount)
    for c in char:
        histogram[int(c/step)] += 1.
    histogram = histogram/len(char)
    histogram = np.ndarray.tolist(histogram)
    return histogram

In [7]:
def reshape_pca(char):
    mfcc = np.array(char[:,0 ])
    mfcc = np.concatenate(mfcc).reshape(1000,24960)
    pca=PCA(n_components = 100, svd_solver='full')
    pca.fit(mfcc)
    mfcc=pca.transform(mfcc)
    return mfcc

In [8]:
def train_test(mfcc, char, n_set):
    train_mfcc, test_mfcc=[],[]
    train_genre, test_genre=[], []

    for i in range(1000):
        if (i+n_set)%10==0:
            test_mfcc.append(mfcc[i])
            test_genre.append(char[i][1])
        else:
            train_mfcc.append(mfcc[i])
            train_genre.append(char[i][1])

    train_mfcc = np.array(train_mfcc)
    test_mfcc = np.array(test_mfcc)
    train_genre = np.array(train_genre)
    test_genre = np.array(test_genre)
    return train_mfcc, train_genre, test_mfcc, test_genre

In [9]:
def classify(train_mfcc, train_genre, test_mfcc, test_genre):
    success=0
    clf=SVC(C=0.0001)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)
        if pred[0]==test_genre[i]:
            success+=1
    return success

In [10]:
def all_hist(char, step, amount):
    hist=[]
    temp_char=[]
    for c in char:
        temp = histogram(step, c[0], amount)
        hist.append((temp, c[1]))
        temp_char.append(temp)
    return hist, temp_char


In [11]:
def more_hist(char, step, amount):
    hist = []
    temp_char=[]
    for c in char:
        tmp = c[0]
        temp = []
        for v in tmp:
            temp+=histogram(step, v, amount)
        temp_char.append(temp)
        hist.append((temp, c[1]))
    return hist, temp_char    

In [12]:
def mode1():
    char = char1()
    mfcc = reshape_pca(char)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char, n_set=0)
    classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [15]:
def classify_mat_2(train_mfcc, train_genre, test_mfcc, test_genre, g):
    succ_gen = np.zeros((10,10))
    clf=SVC(kernel = 'rbf', gamma=g)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)        
        succ_gen[test_genre[i]][pred[0]]+=1
    for i in succ_gen:
        for j in i:
            print "%2.0f" % j,
        print
    success = 0
    for i in range(10):
        success += succ_gen[i][i]
    print success

'''n_mfcc = 15
char = char1(char=n_mfcc)
mfcc = np.array(char[:,0 ])
mfcc = np.concatenate(mfcc).reshape(1000,1248*n_mfcc)
train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char, n_set=0)'''
for i in range(0,10): #-7
    print 10**i, ",", i, ":"
    classify_mat_2(train_mfcc, train_genre, test_mfcc, test_genre, 10**i)

1e-20 , -20 :
 0  0  2  0  0  2  5  0  0  1
 0  8  1  0  0  1  0  0  0  0
 0  0  1  1  0  2  4  0  1  1
 0  0  0  1  0  1  7  1  0  0
 0  0  0  1  0  0  4  3  1  1
 0  3  1  1  0  4  1  0  0  0
 0  0  0  0  0  1  9  0  0  0
 0  0  0  2  0  0  0  8  0  0
 0  0  1  4  0  4  0  0  0  1
 0  0  1  2  0  1  6  0  0  0
31.0
1e-19 , -19 :
 0  0  2  0  0  2  5  0  0  1
 0  8  1  0  0  1  0  0  0  0
 0  0  1  1  0  2  4  0  1  1
 0  0  0  1  0  1  7  1  0  0
 0  0  0  1  0  0  4  3  1  1
 0  3  1  1  0  4  1  0  0  0
 0  0  0  0  0  1  9  0  0  0
 0  0  0  2  0  0  0  8  0  0
 0  0  1  4  0  4  0  0  0  1
 0  0  1  2  0  1  6  0  0  0
31.0
1e-18 , -18 :
 0  0  2  0  0  2  5  0  0  1
 0  8  1  0  0  1  0  0  0  0
 0  0  1  1  0  2  4  0  1  1
 0  0  0  1  0  1  7  1  0  0
 0  0  0  1  0  0  4  3  1  1
 0  3  1  1  0  4  1  0  0  0
 0  0  0  0  0  1  9  0  0  0
 0  0  0  2  0  0  0  8  0  0
 0  0  1  4  0  4  0  0  0  1
 0  0  1  2  0  1  6  0  0  0
31.0
1e-17 , -17 :
 0  0  2  0  0  2  5  0  0  1

KeyboardInterrupt: 

In [15]:
def mode2():
    char = char2()
    mfcc = reshape_pca(char)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char)
    classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [16]:
def mode3(char, step, amount):
    hist, char = all_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [17]:
def mode4(char, step, amount):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [ ]:
for i in range(5,21,5):
    char_gen = chargen(i)
    filename = "values"+str(i)
    np.save(filename, char_gen)

In [ ]:
for i in range(5,6,5):
    filename = "values"+str(i)+".npy"
    char = np.load(filename)
    #print char.shape, char[0].shape, char[0][0].shape, char[0][0][0].shape
    #step = (np.amax(np.amax(np.amax(char))) - np.amin(np.amin(np.amin(char))))/100

In [ ]:
char, step = char3(5)

In [ ]:
hist, charco = more_hist(char,(step*100)/50, 50)

In [ ]:
results=[]
for i in range(1,100):
    if i%10==0: print i,
    results.append((i, mode3(char, (step*100)/i, amount=i)))

print
results.sort(key=lambda x:x[1], reverse=True)
print results[0]

In [ ]:
results=[]
for i in range(5,100):
    if i%10==0: print i,
    results.append((i, mode4(char, (step*100)/i, amount=i)))

print
results.sort(key=lambda x:x[1], reverse=True)
print results
print results[0]

In [ ]:
char, step = char3(15)

In [37]:
#char, step = char3(15)

def mode5(char, step, amount, n_set, gamma):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify_mat(train_mfcc, train_genre, test_mfcc, test_genre, gamma)

def classify2(train_mfcc, train_genre, test_mfcc, test_genre, g):
    success=0
    succ_gen = np.zeros(10)
    clf=SVC(kernel = 'rbf', gamma=g)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)
        if pred[0]==test_genre[i]:
            succ_gen[test_genre[i]] += 1
            success+=1
    print succ_gen, np.mean(succ_gen), success
    return success

def classify_mat(train_mfcc, train_genre, test_mfcc, test_genre, g):
    succ_gen = np.zeros((10,10))
    clf=SVC(kernel = 'rbf', gamma=g)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)        
        succ_gen[test_genre[i]][pred[0]]+=1
    for i in succ_gen:
        for j in i:
            print "%2.0f" % j,
        print
    #return success

'''for j in range(-9,10):
    print "gamma:", 1.4+(j/100.)'''
'''for j in range(10):
    print "set:", j
    for i in range(87,93):
        print str(i),":",'''
mode5(char, (step*100)/90, amount=90, n_set=4, gamma=1.4)

NameError: name 'step' is not defined

In [ ]:
#leave one out
from sklearn.model_selection import LeaveOneOut
'''hist, char2 = more_hist(char, step, 90)

loo = LeaveOneOut()
loo.get_n_splits(char2)'''

i = 0
success = 0

for train_index, test_index in loo.split(char2):
    X_train = np.array([char2[j] for j in train_index])
    X_test = np.array([char2[j] for j in test_index])
    y_train  = np.array([hist[j][1] for j in train_index])
    y_test = np.array([hist[j][1] for j in test_index])
    clf=SVC(kernel = 'linear')
    clf.fit(X_train,y_train)
    test=X_test.reshape(1, -1)
    pred = clf.predict(X_test)
    if pred[0]==y_test:
        success+=1
    if i%10==0:
        print str(i),
    i+=1

print success

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import operator

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

def mode6(char, step, amount, n_set):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify3(train_mfcc, train_genre, test_mfcc, test_genre)

def classify3(train_mfcc, train_genre, test_mfcc, test_genre):
    value = dict([(key, 0) for key in range(15)])
    order = dict([(key, 0) for key in range(15)])
    forest = ExtraTreesClassifier()
    forest.fit(train_mfcc,train_genre)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_mfcc.shape[1]):
        #print("%d. feature %d (%f)" % (f + 1, indices[f]/90, importances[indices[f]]))
        if not importances[indices[f]]<=10**-10:
            print("%d. feature %d (%f)" % (f + 1, indices[f]/90, importances[indices[f]]))
            value[indices[f]/90] +=  1
        else:
            break
    
    value = sorted(value.items(), key=operator.itemgetter(1))
    
    print value

    # Plot the feature importances of the forest
    '''plt.figure()
    plt.title("Feature importances")
    plt.bar(range(100), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(100), indices)
    plt.xlim([-1, 10])
    plt.show()'''

mode6(char, (step*100)/90, amount=90, n_set=4)

NameError: name 'step' is not defined

In [29]:
from sklearn.naive_bayes import GaussianNB

def mode7(char, step, amount, n_set):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify4(train_mfcc, train_genre, test_mfcc, test_genre)

def classify4(train_mfcc, train_genre, test_mfcc, test_genre):
    success=0
    succ_gen = np.zeros(10)
    gnb = GaussianNB()
    y_pred = gnb.fit(train_mfcc,train_genre).predict(test_mfcc)
    #print test_genre == y_pred
    print("Number of mislabeled points out of a total %d points : %d"
      % (test_mfcc.shape[0],(test_genre != y_pred).sum()))

mode7(char, (step*100)/90, amount=90, n_set=4)

NameError: name 'step' is not defined

In [30]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

def mode8(char, step, amount, n_set):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify5(train_mfcc, train_genre, test_mfcc, test_genre)

def classify5(train_mfcc, train_genre, test_mfcc, test_genre):
    clf = AdaBoostClassifier(n_estimators=100)
    scores = cross_val_score(clf, train_mfcc, train_genre)
    print scores.mean()

mode8(char, (step*100)/90, amount=90, n_set=4)  

NameError: name 'step' is not defined

# Results
* char = 5
* step = 18

* **result** = 36


* char = 20
* step = 18

* **result** = 37


* char = 12
* step = 18

* **result** = 37